In [56]:
# 1. Katie Corcoran

In [52]:
# 2.
import pandas as pd
import psycopg2
import warnings
warnings.filterwarnings('ignore')


conn = psycopg2.connect(
    database="loftsvc", 
    user='USER', # Enter your AWS username here
    password='PASSWORD', # Enter your database password here
    host='HOST', # Enter your AWS database server endpoint here
    port= '5432'
)
cursor = conn.cursor()

df = pd.read_sql('''
SELECT
    exceptid,
    record,
    fixstamp
FROM exceptionlog
WHERE fixstamp IS NULL''', conn)
print(df)

    exceptid                                             record fixstamp
0          1  Elizabeth Marsh||Gettysburg|2024-01-17|721|Fau...     None
1          2  Amanda||Hudson|2024-01-17||Mighty cold sucking...     None
2          3       Tracey Thompson|doors|Hudson|2024-01-18|893|     None
3          4  Rhonda Carter|hvac||2024-01-18|268|Aircon soun...     None
4          5  Willie Greene||Charleston|2024-01-19|331|Inter...     None
5          6  Teri Perry|windows|Detroit|2024-01-20|486|Stuc...     None
6          7  Victoria Ward|doors||2024-01-21|345|Door bashe...     None
7          8  Jerry Massey|plumbing|Detroit||431|No hot wate...     None
8          9  John Clark||Antietam|2024-01-21|105|Unexpected...     None
9         10  Shannon Ramos|connectivity||2024-01-22|753|Slo...     None
10        11  Court Boone|connectivity|Charleston|2024-01-22...     None
11        12  Darryl Vazquez|doors|Antietam|2024-01-23|176|D...     None
12        13  Billy Walker||Antietam|2024-01-24|146

In [8]:
# 3.
print('Rhonda Carter: Missing building value. We can use the unit number to look up which building the request belongs to.')
print('Willie Greene: Missing request type. We can compare the description with key words from the request type description.')
print('Jerry Massey: Missing date. We can look at the date of records before and after this one to fill in the date.')
print('Gabriela Carter: Outlets is not a valid request type. We can change anything with "outlets" to an electrical type.')

Rhonda Carter: Missing building value. We can use the unit number to look up which building the request belongs to.
Willie Greene: Missing request type. We can compare the description with key words from the request type description.
Jerry Massey: Missing date. We can look at the date of records before and after this one to fill in the date.
Gabriela Carter: Outlets is not a valid request type. We can change anything with "outlets" to an electrical type.


In [12]:
# 4.
cursor.execute('''
ALTER TABLE servreq
ADD COLUMN IF NOT EXISTS exceptid INTEGER;
''')
conn.commit()

df2 = pd.read_sql('''
    SELECT
        srid,
        exceptid
    FROM servreq
    LIMIT 3
''', conn)
print(df2)

   srid exceptid
0     1     None
1     2     None
2     3     None


/var/folders/44/s9hsvm1943z9xb042mqxxtgc0000gn/T/ipykernel_34212/1136839552.py:8: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df2 = pd.read_sql('''


In [43]:
# 5.
from datetime import datetime

for x in df.index:
    values = df['record'].loc[x]
    exceptid = df['exceptid'].loc[x]
    record = values.split('|')
    reqdescription = record[5]
    
    # Name
    name = record[0].split(' ')
    if (len(name) < 2):
        print(f"Can't fix (One Name): {record}")
        continue
    else:
        tenant = pd.read_sql(f'''
            SELECT
                tenantid
            FROM tenant
            WHERE fname = '{name[0]}'
            AND lname = '{name[1]}';
        ''', conn)
        if len(tenant) == 0:
            print(f"Can't Fix (Unknown Name): {record}")
            continue
        else:
            tenantid = tenant['tenantid'].loc[0]
    # Type
    types = pd.read_sql('''
        SELECT
            rtid,
            typename
        FROM reqtype
    ''', conn, index_col='rtid')
    rtid = types.index[types['typename'] == record[1].lower()]
    if (rtid.empty):
        if ('internet' in reqdescription.lower()):
            rtid = types[types['typename'] == 'connectivity'].index[0]
        elif (rtid == 'outlets'):
            rtid = types[types['typename'] == 'electrical'].index[0]
        else:
            print(f"Can't fix (Unknown Type): {record}")
            continue
    else:
        rtid = rtid[0]
    # Building
    buildings = pd.read_sql('''
        SELECT
            LOWER(bldgname) AS bldgname
        FROM building
    ''', conn)
    bldgid = buildings.index[buildings['bldgname'] == record[2].lower()]
    if (bldgid.empty):
        unit = record[4]
        if (unit == ''):
            print(f"Can't fix (No Unit #): {record}")
            continue
        else:
            bldgid = pd.read_sql(f'''
                SELECT
                    bldgid
                FROM
                (SELECT
                    bldgid,
                    CAST(SPLIT_PART(unitrange, '-', 1) AS integer) AS low,
                    CAST(SPLIT_PART(unitrange, '-', 2) AS integer) AS high
                FROM building) unitrange
                WHERE {unit} >= low AND {unit} <= high
            ''', conn)
            if (len(bldgid) == 0):
                print(f"Can't fix (Bad Unit #): {record}")
                continue
            else:
                bldgid = bldgid.loc[0][0]
    else:
        bldgid = bldgid[0]
    # Date
    try:
        reqdate = datetime.strptime(record[3], '%Y-%m-%d')
    except:
        if (x == 0):
            print(f"Can't Fix (No Date): {record}")
            continue
        else:
            reqdate = df['record'].loc[x-1].split('|')[3]
    # Unit Number
    try:
        unit = int(record[4])
    except:
        print(f"Can't Fix (No Unit #): {record}")
        continue
    # Description
    if (reqdescription == ''):
        print(f"Can't fix (No Description): {record}")
        continue
    
    # All good
    print(tenantid, rtid, bldgid, reqdate, unit, reqdescription, exceptid)

Can't fix (Unknown Type): ['Elizabeth Marsh', '', 'Gettysburg', '2024-01-17', '721', 'Faucet dripping, dripping forever, dripping highly, morosely.']
Can't fix (One Name): ['Amanda', '', 'Hudson', '2024-01-17', '', 'Mighty cold sucking from thermostat; @#$%*&! fan.']
Can't fix (No Description): ['Tracey Thompson', 'doors', 'Hudson', '2024-01-18', '893', '']
455 5 2 2024-01-18 00:00:00 268 Aircon sounds like a ghost lives inside it. 4
593 1 2 2024-01-19 00:00:00 331 Internet running at 270 bits per second. 5
Can't Fix (Unknown Name): ['Teri Perry', 'windows', 'Detroit', '2024-01-20', '486', 'Stuck and injured @#$%*&!, lock @#$%*&! handle will not open.']
504 2 3 2024-01-21 00:00:00 345 Door bashed with an axe bashed with an axe trapped locked inside. 7
255 6 3 2024-01-21 431 No hot water. 11 days. Hope dwindling. 8
Can't fix (Unknown Type): ['John Clark', '', 'Antietam', '2024-01-21', '105', 'Unexpected a/c incursions.']
538 1 7 2024-01-22 00:00:00 753 Slow download of wireless, hopeles

In [53]:
# 6.
from datetime import datetime

for x in df.index:
    values = df['record'].loc[x]
    exceptid = df['exceptid'].loc[x]
    record = values.split('|')
    reqdescription = record[5]
    
    # Name
    name = record[0].split(' ')
    if (len(name) < 2):
        print(f"Can't fix: {record}")
        continue
    else:
        tenant = pd.read_sql(f'''
            SELECT
                tenantid
            FROM tenant
            WHERE fname = '{name[0]}'
            AND lname = '{name[1]}';
        ''', conn)
        if len(tenant) == 0:
            print(f"Can't fix: {record}")
            continue
        else:
            tenantid = tenant['tenantid'].loc[0]
    # Type
    types = pd.read_sql('''
        SELECT
            rtid,
            typename
        FROM reqtype
    ''', conn, index_col='rtid')
    rtid = types.index[types['typename'] == record[1].lower()]
    if (rtid.empty):
        if ('internet' in reqdescription.lower()):
            rtid = types[types['typename'] == 'connectivity'].index[0]
        elif (rtid == 'outlets'):
            rtid = types[types['typename'] == 'electrical'].index[0]
        else:
            print(f"Can't fix: {record}")
            continue
    else:
        rtid = rtid[0]
    # Building
    buildings = pd.read_sql('''
        SELECT
            LOWER(bldgname) AS bldgname
        FROM building
    ''', conn)
    bldgid = buildings.index[buildings['bldgname'] == record[2].lower()]
    if (bldgid.empty):
        unit = record[4]
        if (unit == ''):
            print(f"Can't fix: {record}")
            continue
        else:
            bldgid = pd.read_sql(f'''
                SELECT
                    bldgid
                FROM
                (SELECT
                    bldgid,
                    CAST(SPLIT_PART(unitrange, '-', 1) AS integer) AS low,
                    CAST(SPLIT_PART(unitrange, '-', 2) AS integer) AS high
                FROM building) unitrange
                WHERE {unit} >= low AND {unit} <= high
            ''', conn)
            if (len(bldgid) == 0):
                print(f"Can't fix: {record}")
                continue
            else:
                bldgid = bldgid.loc[0][0]
    else:
        bldgid = bldgid[0]
    # Date
    try:
        reqdate = datetime.strptime(record[3], '%Y-%m-%d')
    except:
        if (x == 0):
            print(f"Can't Fix: {record}")
            continue
        else:
            reqdate = df['record'].loc[x-1].split('|')[3]
    # Unit Number
    try:
        unit = int(record[4])
    except:
        print(f"Can't Fix: {record}")
        continue
    # Description
    if (reqdescription == ''):
        print(f"Can't fix: {record}")
        continue
    
    # All good
    cursor.execute(f'''
        INSERT INTO servreq (tenantid, rtid, bldgid, reqdate, unit, reqdescription, exceptid)
        VALUES ({tenantid}, {rtid}, {bldgid}, '{reqdate}', {unit}, '{reqdescription}', {exceptid});
    ''')
    cursor.execute(f'''
        UPDATE exceptionlog
        SET fixstamp = CURRENT_TIMESTAMP
        WHERE exceptid = {exceptid}
    ''')
    conn.commit()
    print(f"Added: {tenantid, rtid, bldgid, reqdate, unit, reqdescription, exceptid}")
    

Can't fix: ['Elizabeth Marsh', '', 'Gettysburg', '2024-01-17', '721', 'Faucet dripping, dripping forever, dripping highly, morosely.']
Can't fix: ['Amanda', '', 'Hudson', '2024-01-17', '', 'Mighty cold sucking from thermostat; @#$%*&! fan.']
Can't fix: ['Tracey Thompson', 'doors', 'Hudson', '2024-01-18', '893', '']
Added: (455, 5, 2, datetime.datetime(2024, 1, 18, 0, 0), 268, 'Aircon sounds like a ghost lives inside it.', 4)
Added: (593, 1, 2, datetime.datetime(2024, 1, 19, 0, 0), 331, 'Internet running at 270 bits per second.', 5)
Can't fix: ['Teri Perry', 'windows', 'Detroit', '2024-01-20', '486', 'Stuck and injured @#$%*&!, lock @#$%*&! handle will not open.']
Added: (504, 2, 3, datetime.datetime(2024, 1, 21, 0, 0), 345, 'Door bashed with an axe bashed with an axe trapped locked inside.', 7)
Added: (255, 6, 3, '2024-01-21', 431, 'No hot water. 11 days. Hope dwindling.', 8)
Can't fix: ['John Clark', '', 'Antietam', '2024-01-21', '105', 'Unexpected a/c incursions.']
Added: (538, 1, 7,

In [54]:
# 7.
df3 = pd.read_sql('''
SELECT
	srid,
    tenantid,
    rtid,
    bldgid,
    reqdate,
    unit,
    exceptid
FROM servreq
WHERE exceptid IS NOT NULL
''', conn, index_col='srid')
print(df3)

      tenantid  rtid  bldgid     reqdate  unit  exceptid
srid                                                    
116        455     5       2  2024-01-18   268         4
117        593     1       2  2024-01-19   331         5
118        504     2       3  2024-01-21   345         7
119        255     6       3  2024-01-21   431         8
120        538     1       7  2024-01-22   753        10
121        116     2       0  2024-01-23   176        12
122        495     3       4  2024-01-25   413        14


In [55]:
# 8.
df4 = pd.read_sql('''
SELECT
    exceptid,
    record,
    fixstamp
FROM exceptionlog
WHERE fixstamp IS NOT NULL
''', conn, index_col='exceptid')
print(df4)

                                                     record  \
exceptid                                                      
4         Rhonda Carter|hvac||2024-01-18|268|Aircon soun...   
5         Willie Greene||Charleston|2024-01-19|331|Inter...   
7         Victoria Ward|doors||2024-01-21|345|Door bashe...   
8         Jerry Massey|plumbing|Detroit||431|No hot wate...   
10        Shannon Ramos|connectivity||2024-01-22|753|Slo...   
12        Darryl Vazquez|doors|Antietam|2024-01-23|176|D...   
14        Laurie Lopez|electrical||2024-01-25|413|Hopele...   

                           fixstamp  
exceptid                             
4        2024-10-31 19:50:58.543554  
5        2024-10-31 19:51:03.771218  
7        2024-10-31 19:51:04.648846  
8        2024-10-31 19:51:05.511218  
10       2024-10-31 19:51:05.975149  
12       2024-10-31 19:51:06.643901  
14       2024-10-31 19:51:07.146334  
